In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine

In [2]:
username = 'admin'
password = 'nX0d4tuvxL3w5hMjJRfz'
host = 'ai-engine.cluster-cla9jbugaywl.eu-west-2.rds.amazonaws.com'
database_name = 'superleague_clubs'

connection_url = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'

def insert_into_db(df, table_name) : 

    engine = create_engine(connection_url)
    df.to_sql(table_name, con=engine, if_exists='replace', index=False) 
    return 'Data inserted successfully'

In [3]:
club = 'inter-mailand'
club_id = '46'
url = 'https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id='

In [4]:
data = []
headers = ['shirt_number', 'playername', 'player_url','mainposition', 'position', 'birthdate', 'country', 'club_name', 'market_value', 'season']

startyear = 1992
endyear = 2023
for year in range(startyear, endyear+1) : 

    cluburl = url + str(year)
    print(cluburl)

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") 

    driver = webdriver.Chrome() 
    driver.get(cluburl)

    driver.implicitly_wait(5) 
    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, "lxml")

    table = soup.select('.responsive-table > .grid-view > .items > tbody')[0]

    for row in table.find_all(True, {"class": ["odd", "even"]}):

        try : 
            rowdata = []

            mainposition = row.find('td')['title']
            shirt_number = row.find('div', class_='tm-shirt-number').text.strip()
            player_name = row.find('td', class_='posrela').find_all('tr')[0].td.img['title']
            player_url = row.find('td', class_='posrela').find('tr', class_='data-link')['data-link']
            position_element = row.find('td', class_='posrela')
            position = position_element.find_all('tr')[1].text if position_element else '-'
            birthdate_element = row.find_all('td', class_='zentriert')[1]
            birthdate = birthdate_element.text.strip().split('(')[0].strip() if birthdate_element else '-'
            country_element = row.find_all('td', class_='zentriert')[2].img
            country = country_element['title'] if country_element else '-'
            club_element = row.find_all('td', class_='zentriert')[3].a
            club_name = club_element['title'] if club_element else '-'
            market_value_element = row.find('td', class_='rechts hauptlink').a
            market_value = market_value_element.text.strip() if market_value_element else '-'

            rowdata.append(shirt_number)
            rowdata.append(player_name)
            rowdata.append(player_url)
            rowdata.append(mainposition)
            rowdata.append(position)
            rowdata.append(birthdate)
            rowdata.append(country)
            rowdata.append(club_name)
            rowdata.append(market_value)
            rowdata.append(str(year)+'-'+str(year+1))

        except : 
            continue

        data.append(rowdata)

clubdf = pd.DataFrame(data, columns=headers)    

https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1992
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1993
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1994
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1995
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1996
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1997
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1998
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=1999
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=2000
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=2001
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=2002
https://www.transfermarkt.com/inter-mailand/startseite/verein/46?saison_id=2003
https://www.transfermarkt.com/inter-mail

In [5]:
insert_into_db(clubdf, f'$transfermarkt_{club}')

'Data inserted successfully'